In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/home/kf7mxe/anaconda3/envs/time-series-data-mining/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# import data.tsv file and create a dataframe

df = pd.read_csv('data.tsv', sep='\t')

# open file data_info.txt and read the first line

sequence_length = None
with open('data_info.txt', 'r') as f:
    line = f.readline()
    split_line_on_collen = line.split(':')
    sequence_length = int(split_line_on_collen[1])
    


train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)


# drop the collumn Y from the dataframe

train_x = train_df.drop(['Y'], axis=1).iloc[1:]
train_y = train_df['Y']

test_x = test_df.drop(['Y'], axis=1)


test_y = test_df['Y']


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, input, seq_len):
        self.input = input
        self.seq_len = seq_len
    def __getitem__(self, item):
        return input[item:item+self.seq_len], input[item+self.seq_len]
    def __len__(self):
        return len(self.input) - self.seq_len



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


input_size = 3
num_classes = 2
hidden_size = 256
num_layers = 2

class SitUpDetector(nn.Module):
    def __init__(self,input_size, num_classes, hidden_size, num_layers):
        super(SitUpDetector, self).__init__()
        self.hidden_dim = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)

        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        return hidden

model = SitUpDetector(input_size, num_classes, hidden_size, num_layers).to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1000):
    for batch_idx, (data, target) in enumerate(zip(train_x, train_y)):
        print(data)
        print(type(data))

        data = torch.tensor(data).float().to(device)
        target = torch.tensor(target).float().to(device)

        optimizer.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss.backward()

        optimizer.step()



        # data = data.to(device=device)
        # targets = targets.to(device=device)

        # # forward
        # scores = model(data)
        # loss = criterion(scores, targets)

        # # backward
        # optimizer.zero_grad()
        # loss.backward()

        # # gradient descent or adam step
        # optimizer.step()


# test the model

with torch.no_grad():
    output = model(test_x)
    print(output)
    print(test_y)


# save the model

torch.save(model.state_dict(), 'sit_up_detector.pth')






0
<class 'str'>


TypeError: new(): invalid data type 'str'